# 전문가 답변 & 기간 설정 크롤링

In [1]:
import time
import urllib.request, urllib.parse
from bs4 import BeautifulSoup
import csv
import re
import pandas as pd
import time
import numpy as np
import requests
import lxml.html

def naver_crawler(quest, year):
    # 검색키워드, 검색 날짜 범위 지정 
    query = quest
    period_from =  str(year) + '.01.01.'
    period_to =  str(year) + '.12.31.'
    
    # 주소 생성 
    naver_front = 'https://kin.naver.com/search/list.nhn?query='
    period_from = '&period=' + str(period_from)
    period_to = '%7c' + str(period_to)
    kin_url = naver_front + query + period_from + period_to + '&page={}'
    
    # 크롤링 내역 담을 리스트 생성 
    ques = []
    answer = []
    date = []
    
    # 각 범위주소마다 페이지 범위 넘겨가며 크롤링 진행 
    page = 0
    while True:
        page += 1 
        # 점속 컨택 
        # time.sleep(0.1)
        res = requests.get(kin_url.format(page)) 
        root = lxml.html.fromstring(res.text)
        css = root.cssselect('.basic1 a') # 링크가 있는 루트 
     
        # 각 질문마다의 링크 리스트에 담음. 
        links = []
        # 링크가 이상한 경우가 있어 https 주소로 되어있는 경우만 가져옴.
        for i in css:
            if 'https:' in i.attrib['href']:
                links.append(i.attrib['href'])
            else:
                next
                
        # 각 링크마다의 제목, 답변, 게시일 담음.
        for link in links:
            #time.sleep(0.1)
            res_links = requests.get(link)
            root_links = lxml.html.fromstring(res_links.text) 
            try:
                ques.append(root_links.cssselect('.c-heading__content')[0].text_content())
            except IndexError:
                ques.append('내용없음')
            # time.sleep(0.1)
            try:
                answer.append(re.sub(r'\s+', '',root_links.cssselect('._endContentsText')[0].text_content())
            except IndexError:
                answer.append(np.NaN)
            # time.sleep(0.1)
            try:
                date.append(root_links.cssselect('.c-userinfo__info')[0].text_content().strip()[3:])
            except IndexError:
                date.append(np.NaN)  # 게시글이 없는 경우가 있음 결측값 처리 
        if len(links) == 0: 
            break # links가 없다. 즉 페이지에 질문이 없는 경우는 페이지가 끝난 것이므로 멈춤
        
    DF = pd.DataFrame({'Question':ques,'Answer':answer,'Date':date}) 
    return DF 

In [52]:
def naver_crawler(quest, year):
    # 검색키워드, 검색 날짜 범위 지정 
    query = quest
    period_from =  str(year) + '.01.01.'
    period_to =  str(year) + '.12.31.'
    
    # 주소 생성 
    naver_front = 'https://kin.naver.com/search/list.nhn?query='
    period_from = '&period=' + str(period_from)
    period_to = '%7c' + str(period_to)
    kin_url = naver_front + query + period_from + period_to + '&page={}'
    
    # 크롤링 내역 담을 리스트 생성 
    ques = []
    answer = []
    date = []
    
    # 각 범위주소마다 페이지 범위 넘겨가며 크롤링 진행 
    page = 0
    while True:
        page += 1 
        # 점속 컨택 
        # time.sleep(0.1)
        res = requests.get(kin_url.format(page)) 
        root = lxml.html.fromstring(res.text)
        css = root.cssselect('.basic1 a') # 링크가 있는 루트 
     
        # 각 질문마다의 링크 리스트에 담음. 
        links = []
        # 링크가 이상한 경우가 있어 https 주소로 되어있는 경우만 가져옴.
        for i in css:
            if 'https:' in i.attrib['href']:
                links.append(i.attrib['href'])
            else:
                next
                
        # 각 링크마다의 제목, 답변, 게시일 담음.
        for link in links:
            #time.sleep(0.1)
            res_links = requests.get(link)
            root_links = lxml.html.fromstring(res_links.text) 
            try:
                ques.append(root_links.cssselect('.c-heading__content')[0].text_content())
            except IndexError:
                ques.append('내용없음')
            # time.sleep(0.1)
            try:
                answer.append(root_links.cssselect('._endContentsText')[0].text_content())
            except IndexError:
                answer.append(np.NaN)
            # time.sleep(0.1)
            try:
                date.append(root_links.cssselect('.c-userinfo__info')[0].text_content()[3:])
            except IndexError:
                date.append(np.NaN)  # 게시글이 없는 경우가 있음 결측값 처리 
        if len(links) == 0: 
            break # links가 없다. 즉 페이지에 질문이 없는 경우는 페이지가 끝난 것이므로 멈춤
        
    DF = pd.DataFrame({'Question':ques,'Answer':answer,'Date':date}) 
    return DF 

In [26]:
naver_2002 = naver_crawler('장애인', 2002)
naver_2002

,Question,Answer,Date
0,"\n\t\t\t\t\t\t아주 어렸을 때부터 정말 궁금했습니다..시각장애인들, 즉 ...","'꿈'이라는건 정확히 알려진 바가 없으나""잠재의식(공포,과거의 경험,소망),이 깊지...",2002.10.29
1,\n\t\t\t\t\t\t장애인들이 개선 되기를 원하는 점은 무엇일까요..?여러가지...,안녕하세요 균이 입니다 *^^*장애인들이 가장 개선 되기 원하는 것은 장애인에 대...,2002.11.13
2,\n\t\t\t\t\t\t제가 아침에 환청을 듣고 잠에서 깼거든요;너무 무서워서 생...,먼저 환청이라는 것은 주위에 사람이나 소리나는 사물이 아무것도 없는데도 어떤 소리나...,2002.12.31
3,\n\t\t\t\t\t\tTV 연속극에서 시각장애인이 눈물을 흘리는 장면이 나와 이...,눈물은 기본적인 눈물과 반사적인 눈물로 나누어 볼 수 있는데 우리가 일반적으로 아는...,2002.12.25
4,\n\t\t\t\t\t\t그냥 비장애인들도 하는 경기 말구요...장애인들만 하는 경...,1996년 애틀란타 장애인올림픽에서 시범종목이었던 휠체어 럭비와 요트 경기가 정식 ...,2002.10.30
...,...,...,...
341,\n\t\t\t\t\t\t요즘은 애완동물을 키우는 사람들이 참 많잖아요.그래서인지 ...,[애완동물의 대중교통 이용]제21조 승객의 금지행위 승객운송에 사용되는 자동차를 이...,2002.09.09
342,\n\t\t\t\t\t\t학교 숙제인데 국가가 나를 위해 해주기를 바라는 일과 내가...,가.옛날의 국가와 오늘날의 국가① 나라가 하는 일은 시대에 따라 변함.㉠옛날 : 산...,2002.10.21
343,\n\t\t\t\t\t\t제가.무역회사 경린데여..도대체 연말소득공제가 뭔지.공제해...,제 전공이라서 답변을 드립니다~!근로소득자임을 가정하고 답변을 드립니다~!1. 연말...,2002.11.27
344,"\n\t\t\t\t\t\t윌렌스키와 르보가 제시한 보충적, 제도적 복지이념 혹은 복...",전통사회에서 산업사회로 변화됨에 따라 사회가 시장경쟁체제로 전환되면서 생산과 소비가...,2002.10.22


In [32]:
naver_2002.to_csv('2002장애인.csv')

In [27]:
naver_2003 = naver_crawler('장애인', 2003)
naver_2003

,Question,Answer,Date
0,\n\t\t\t\t\t\t세종대왕이 시각장애인이라는 이야기를 친구에게 들었어요그런데...,지난 천년 동안 가장 위대한 인물과 가장 자랑스러운 일에 관한 여론조사에서 우리나라...,2003.03.16
1,\n\t\t\t\t\t\t저는 장애인이며 급여소득자입니다.연말정산을 하려고 하는데의...,님께서는 일반의료비공제와 더불어 (사용액-총급여의 3% .한도500만원)일반의료비의...,2003.12.24
2,\n\t\t\t\t\t\t요즈음에는 장애인이란 단어보다 장애우란 말을 더 널리 쓰는...,최근 동아일보 독자란에 '장애인'이 아닌 '장애우'란 표현을 쓰자는 주장이 있었습니...,2003.08.30
3,\n\t\t\t\t\t\t장애인을 왜 차별하는 걸까요 ? 또 장애인 차별의 사례와 ...,현대사회에서는 서로에 대한 경쟁과 집단 이기주의 등으로 남들을 많이 생각하지 않고 ...,2003.11.09
4,\n\t\t\t\t\t\t제가 아는 분이 5년전 허리수술을 하셔서 인공뼈까지 넣으셨...,허리 수술하신 분께 힘내시라고 먼저 말씀드립니다. 등록절차 ...,2003.06.04
...,...,...,...
3125,내용없음,NaN,NaN
3126,내용없음,NaN,NaN
3127,내용없음,NaN,NaN
3128,내용없음,NaN,NaN


In [33]:
naver_2003.to_csv('2003장애인.csv')

In [28]:
naver_2004 = naver_crawler('장애인', 2004)
naver_2004

,Question,Answer,Date
0,\n\t\t\t\t\t\t동생이 장애인이라 동생명의로 차량을 구입하려는데 장애인차량...,장애인 차량은 등급에 따라 혜택이 조금 다릅니다.승합차의 경우에는 RV 차량으로 이...,2004.01.05
1,"\n\t\t\t\t\t\t점자블럭 하구 안내판, 장애인용 주차장, 리프트, 장애인용...",지금 적으신 것들은 전부 장애인을 위한 편의시설에 포함되는 것들입니다.편의시설중에서...,2004.10.17
2,\n\t\t\t\t\t\t아버지가 뜻하지 않은 교통사고로 뇌수술의 2차례나 받으시고...,예전에 hoas12님이 답변한 내용인데 어디서 가져온 자료인지 모르나 잘 정리가 되...,2004.07.26
3,"\n\t\t\t\t\t\t봉사활동을 하거나,어떤장소에서 나의 도움이 필요한 장애인을...",http://www.freeget.net/common/etiquette4.html#...,2004.09.23
4,\n\t\t\t\t\t\t저의 형이 장애인인데 가족의 경우에 장애인 경우 가스차를 ...,장애인 등급이 있으면 LPG 를 연료로 하는 승용차를 구입할 수 있습니다.. 장애 ...,2004.05.14
...,...,...,...
4835,\n\t\t\t\t\t\t답변좀 부탁드릴께요...제가 장애등급 6급인데요..요번에 ...,신용불량 상태와 장애인 등급과느느 전혀 별개의 사안입니다. 하지만 신용조회를 하면 ...,2004.11.10
4836,\n\t\t\t\t\t\t장애인 등록에 관해서 무러볼께 있습니다..발이 평발이 아닌...,님의 질문 상태로 봐서는 장애등록은 불가능 할 것으로 판단이 됩니다..다리의 장애로...,2004.07.15
4837,\n\t\t\t\t\t\t방송국들은 장애인 올림픽에 대해 관심을 가져 달라고 하면서...,우리나라의 인식변화가 아직도 19세기 수준이라는 것과....가장 중요한 것은.......,2004.09.23
4838,\n\t\t\t\t\t\t제가 보조개가 있는데요 막 남학생들이 아는 형이 보조개있는...,아뇨아뇨>ㅁ쌍꺼풀은 누구나있을수있는거구요보조개는 웃을때볼을 예쁘게톡톡튀게해주는거자노...,2004.10.03


In [34]:
naver_2004.to_csv('2004장애인.csv')

In [29]:
naver_2005 = naver_crawler('장애인', 2005)
naver_2005

,Question,Answer,Date
0,\n\t\t\t\t\t\t 신체적 장애인 - 신체 기능의 장애인 : - ...,이미 잘 알고 계시네요. 현행 장애인복지법에 근거한 등록장애인의 분류는 크게 신체적...,2005.06.26
1,\n\t\t\t\t\t\t장애인차량구입관련해서 질문하겠습니다. 1. 장애인본인단독으...,질문 순서에 따라 설명을 드리겠습니다 1. 장애인의 경우 이동권을 보호하는 차원에서...,2005.06.09
2,\n\t\t\t\t\t\t제가 5급 장애인증을 발급받아 아버지와 공동명의로 LPG차...,안녕하세요.저또한 지금 장애인혜택을 받아서 LPG 차량을 구매하려고 준비중이라몇자 ...,2005.11.23
3,\n\t\t\t\t\t\t장애인 6급판정을 받았는뎅장애인용 엘피지차를 살수 있는지 ...,장애인이시면 등급에 관계없이 장애인용 LPG차량을 구입하실 수 있습니다. 하지만 ...,2005.05.06
4,\n\t\t\t\t\t\t장애인 6급혜택자세히 적어주세요\n\t\t\t\t\t,1.장애인 자립자금 대여저소득가구의 장애인 가구주 또는 가구주의 배우자인 장애인(수...,2005.01.09
...,...,...,...
3965,내용없음,NaN,NaN
3966,내용없음,NaN,NaN
3967,내용없음,NaN,NaN
3968,내용없음,NaN,NaN


In [35]:
naver_2005.to_csv('2005장애인.csv')

In [30]:
naver_2006 = naver_crawler('장애인', 2006)
naver_2006

,Question,Answer,Date
0,"\n\t\t\t\t\t\t어머니가 3급 장애인이신데, 제가 이번에 부산에 내려가거든...",정당한방법은 아닙니다.이 장애인할인은 1~3급장애인 그 보호자를 포함해서만 되는것이...,2006.10.14
1,\n\t\t\t\t\t\t복지행정과에 다니는 학생입니다.장애인에 관한 신문기사 스크...,장애인에 관한 신문기사 스크랩을 하기 위해 신문을 구독할 수는 있지만정보의 바다인 ...,2006.09.10
2,\n\t\t\t\t\t\t장애인 고용촉진법의 뜻을 알려주세요.. \n\t\t\t\t\t,장애인 고용촉진법의 정확한 공식명칭은 [장애인고용촉진 및 직업재활법]입니다. 이 법...,2006.03.14
3,\n\t\t\t\t\t\t허리디스크로 인한 장애인증이 있으면 행정고시에 응시할때 장...,아래 공고문을 보고요... 더 상세는 출처로 가면 됩니다.. 2006년도 행정고등고...,2006.11.23
4,\n\t\t\t\t\t\t무엇이 있는지 상세히 알려주세요.......참고로 전 뇌병...,1.장애인 자립자금 대여저소득가구의 장애인 가구주 또는 가구주의 배우자인 장애인(수...,2006.05.23
...,...,...,...
8315,\n\t\t\t\t\t\t장인어른께선 3급장애인 입니다.주소지를 부산에 두고 계시구...,"중고차량을 구입하고자 한다면, 운전면허증 상관 없이 가능합니다.(일시불)그러므로 굳...",2006.08.05
8316,\n\t\t\t\t\t\t오늘이 장애인의 날 이잔아요 그래서 6교시 끝나고 렛츠댄스...,원고지 10장이면 많은것 같지만 실제 분량은 적습니다.감상문이라 했으니비디오의 대강...,2006.04.20
8317,내용없음,NaN,NaN
8318,내용없음,NaN,NaN


In [36]:
naver_2006.to_csv('2006장애인.csv')

In [39]:
naver_2007 = naver_crawler('장애인', 2007)
naver_2007

,Question,Answer,Date
0,\n\t\t\t\t\t\t제가 1톤 포터를 살때 아버님이 장애인이셔서 저하구 아버님...,주소를 분리하여 세금이 부과되었으면 세금을 납부해야 합니다. 부과된 세금은 납부하시...,2007.04.04
1,\n\t\t\t\t\t\t장애인 차량 구입 대하여. 현재 할머님이 중풍으로 장애인 ...,1.차량 구입시 저와 주소가 같이 되지 않아도 아버님 명의로 차량 구입이 가능하다고...,2007.01.17
2,\n\t\t\t\t\t\t안녕하세요 저는 시각장애인 4급이구요 일반 인문계 고등학교...,안녕하세요? 분당에서 수시학원 운영중입니다.일단 올해 확정된 모집요강은 아직 나...,2007.04.10
3,\n\t\t\t\t\t\t4급 장애인( 모친 - 운전면허증 무) 과 등본상 같이 기...,1. 4급으로 lpg 차량구입시 세금 혜택이 전혀 없나요?4급이면 공체만 면제가 됩...,2007.08.22
4,\n\t\t\t\t\t\t장애인 차량 등록과 관련하여 문의드립니다. 현상황1. 장애...,nam720723 님이 잘 모르시는 말씀을 하셨네요 질문1. 장애인 차량을 구매할...,2007.08.31
...,...,...,...
4735,내용없음,NaN,NaN
4736,내용없음,NaN,NaN
4737,내용없음,NaN,NaN
4738,내용없음,NaN,NaN


In [40]:
naver_2007.to_csv('2007장애인.csv')

In [53]:
naver_2008 = naver_crawler('장애인', 2008)
naver_2008

,Question,Answer,Date
0,\n\t\t\t\t\t\t 거제시 장애인 종합 복지관 싸이트 알려줄래요~? (장애인...,안녕하세요...^^* 거제시에 있는 장애인복지시설 안내입니다.시설종별시설명소재지법인...,2008.09.08
1,\n\t\t\t\t\t\t 장애인핸드레일 최소높이를 알고싶어서 문의드립니다. 그...,더 낮출 수 없습니다~~^^* 지금 도면도 낮은데요~~더 높여야 합니다~~^^* ★...,2008.11.08
2,\n\t\t\t\t\t\t3급 지체장애 가지신 어머니 모시고 있던 사람입니다.어머니...,궁금한 것은 LPG차량입니까?LPG차량인 경우는 장애인 어머니와 공동명의를 해지하...,2008.01.15
3,\n\t\t\t\t\t\t목발로 보행하는 지체장애인입니다.휄체어 장애인에 비해 활동...,세상 어디서나 누구에게나 이동권이 동등하게 보장되는 세상을 기원하며 몇자 적습니다....,2008.10.12
4,\n\t\t\t\t\t\t장애인의 인권좀 알려주세요 내공 30드릴게요\n\t\t\t...,장애인 인권 보호 방법이라....인권 보호라기 보다는 인권 존중이라는 차원이 더 옳...,2008.10.08
...,...,...,...
4365,\n\t\t\t\t\t\t저희언니가 장애1급언어지능지체장애인데.건강한편이고 문제가 ...,메리츠화재 FC염찬용입니다.언니 생각해서 보험가입하실려는 생각 대견스럽습니다.제 고...,2006.03.26
4366,내용없음,NaN,NaN
4367,내용없음,NaN,NaN
4368,내용없음,NaN,NaN


In [54]:
naver_2008.to_csv('2008장애인.csv')

In [55]:
naver_2009 = naver_crawler('장애인', 2009)
naver_2009

,Question,Answer,Date
0,\n\t\t\t\t\t\t이번에 대학들어가는 새내기입니다. 다름이 아니라 제가 한쪽...,뉴스를 잘못보신거 같네요.. 장애우 공채시험은 현재와 동일합니다..(안그러면 현재 ...,2009.10.26
1,"\n\t\t\t\t\t\t지식인 보니까 기초생활수급자,차상위계증 아니다면 받을수가 ...",본래에는 장애인 수당은 기초생활수급권자에게만 지급되었었습니다.하지만 요근래에 차상위...,2009.03.08
2,\n\t\t\t\t\t\t아는 분이 4.5t 차량을 이용해서 매일 서울에 올라가시는...,장애인 고속도로 통행료 할인에서 대형차는 해당이 되지 않습니다. 41.고속도로 통행...,2009.02.14
3,\n\t\t\t\t\t\t차상위 장애인 3급인대요 헤택을 자세히 알고싶습니다내공 5...,님의 안타까운 질문에 위로도 할겸 답변을 드립니다. 님 힘내세요. 모든 장애인도 님...,2009.09.02
4,\n\t\t\t\t\t\t2008년 연말정산때 아버지 (58년) 장애인 공제를 신청...,답변 아버님의 근로소득금액을 100만원 이상(이하)이면(불가) 가능 입니다. 장...,2009.03.28
...,...,...,...
5745,내용없음,NaN,NaN
5746,내용없음,NaN,NaN
5747,내용없음,NaN,NaN
5748,내용없음,NaN,NaN


In [56]:
naver_2009.to_csv('2009장애인.csv')

In [57]:
naver_2010 = naver_crawler('장애인', 2010)
naver_2010

,Question,Answer,Date
0,\n\t\t\t\t\t\tㅜ.ㅜ 이거 좀 부탁드릴게요틀리면 제대로 고쳐주시고 빈칸은...,안녕하세요 저는 청각 장애인입니다. <가로열쇠>1 : 소리가 아닌 손짓을 이용해서 ...,2010.04.20
1,\n\t\t\t\t\t\t장애인 전환교육에 대한 기대효과 및 향후 대책 방안 모색에...,장애인 전환교율이 어떤건진 모르겠으나통합교육이면 장애인 학교같은걸 말하는거 같은데장...,2010.01.07
2,\n\t\t\t\t\t\t3형제가 있습니다첫째 누나(52살)둘째 남동생(50살)장애...,# 채무자와 어떤 관계 이신지 모르겠으나 계속 질문을 하실 것 같아 답변 드립니다....,2010.10.14
3,\n\t\t\t\t\t\t제가 4학년인데요..... 학교에서 장애인 날 신문을 주었...,안녕하세요. 전체를 볼 수 없으니 잘 모르겠네요. <가로열쇠>3.혼자서 공부하기 어...,2010.04.16
4,"\n\t\t\t\t\t\t안녕하세요..,저는 청각장애인2급인데요...중학생1학년인데...","기존의 장애인, 노인을 위한 교통지원 제도는는 지하철만 해당됩니다.. 왜냐하면.. ...",2010.06.15
...,...,...,...
4835,\n\t\t\t\t\t\t○ 현재상황- 싼타페cm 2006년2월 구매한 차량을 아버...,질문- 기존차량을 아버님에게 명의이전할경우... 이전받은사람은 취등록세 내야하나요?...,2010.05.26
4836,\n\t\t\t\t\t\t장애인 등급에 따라서 등록세가 면제 되는것으로 아는데요 제...,안녕하세요. 장애인은 1급부터 6급까지 인데 등록세 취득세 자동차세 등은 1~3급(...,2010.04.21
4837,\n\t\t\t\t\t\t현재 상황. 우선 아버지 장애인 5급 (무면허)+ 동생 ...,참으로 딱하군요! 제 인사를 간략하게 올리면 20년차 자동차매매업을 운영하고 있습니...,2010.03.23
4838,\n\t\t\t\t\t\t장애인근로자를 위한 자동차 구입 융자제도가 없어진건가요.....,"한국장애인고용공단에서 관장하던 ""장애인근로자 자동차구입 자금"" 사업이 2010년 보...",2010.06.03


In [58]:
naver_2010.to_csv('2010장애인.csv')

In [59]:
naver_2011 = naver_crawler('장애인', 2011)
naver_2011

,Question,Answer,Date
0,\n\t\t\t\t\t\t저는 일반인으로 살다가 장애인이 되버린 지적 장애3급인 장...,1자동차를 취득하며는 자동차새와 취득새를 내야하는데 이것을 안내도 되는걸로 알고 있...,2011.07.13
1,"\n\t\t\t\t\t\t학교에서 하라는 건대 알려주세에요, 1.청각장애인, 시각장...",저는 시각장애인으로써 답변해 드리겠습니다.저는 장애인복지관을 많이 다니면서시각장애인...,2011.10.18
2,\n\t\t\t\t\t\t우리학교는 딴학교랑 다르더라구여~ㅠㅠㅠㅠㅠㅠ제발다풀어주세여...,<가로열쇠>1.전기로 편리하게 움직일 수 있도록 만든 휄체어: =전동휠체어2.청각장...,2011.04.21
3,\n\t\t\t\t\t\t장애인이 빈곤상태에 놓여있을떄 가장 필요한 장애인 권리는 ...,"안녕하세요. 1998년 12월 9일 선포된 장애인 인권헌장에 따르면,장애인이 빈곤상...",2011.01.09
4,\n\t\t\t\t\t\t1. 1926년 재생원 맹아부 교사였던 박두성이 만든 한글...,1. 1926년 재생원 맹아부 교사였던 박두성이 만든 한글점자 체계=훈맹정음 12...,2011.12.13
...,...,...,...
530,"\n\t\t\t\t\t\t영화를 찾고있는데 외국영화이구요, 배우이름을 잘 모르겠어서...",여인의 향기 - 1992년작 주연배우 알파치노가 시각장애인인데 영화 중간에 멋드러...,2011.07.08
531,"\n\t\t\t\t\t\t영화를 찾고있는데 외국영화이구요, 배우이름을 잘 모르겠어서...",여인의 향기 - 1992년작 주연배우 알파치노가 시각장애인인데 영화 중간에 멋드러...,2011.07.08
532,\n\t\t\t\t\t\t단순히 장애인 돌봐주는건가요? 채용정보에는 보육교사 자격증...,"보육이란, 만6세미만의 어린아이들을 보호하고 교육하는 것을 말합니다. 장애인 보육교...",2011.12.29
533,\n\t\t\t\t\t\t어머니가 시각장애인 1급으로 등록되있으신데제가 1600cc...,등록은 가능합니다.하지만 전제 조건이 있습니다. 어머니랑 동일 세대여야 합니다.공동...,2011.06.10


In [60]:
naver_2011.to_csv('2011장애인.csv')

In [67]:
naver_2012 = naver_crawler('장애인', 2012)
naver_2012 

,Question,Answer,Date
0,\n\t\t\t\t\t\tOX 1 . 세계적으로 시각장애인이 사용하는 지팡이 의 색...,1 . 세계적으로 시각장애인이 사용하는 지팡이 의 색갈은 노란색이다 X 2. 시...,2012.04.23
1,\n\t\t\t\t\t\t안녕하세요 일반사람들이 시각장애인에게배려하는것이 뭐가 잇나...,"제가 시각장애인입니다..ㅋㅋ 전 시각장애 4급이구요,ㅋ 시력은 나안으로 0.1정도 ...",2012.03.13
2,\n\t\t\t\t\t\t제가 기초수급자로 장애수당을 받고 있는데요. 장애인 연금에...,1. 지급대상 18세 이상의 등록한 장애인 중 3~6급의 장애등급을 가진 자로 국민...,2012.10.10
3,\n\t\t\t\t\t\t 남편이 이번에 장애2급 판정을 받았습니다.장애인등록증을 ...,안녕하세요. 힘이 되는 평생친구 보건복지부입니다.귀하의 건강과 행복을 기원합니다. ...,2012.06.12
4,\n\t\t\t\t\t\t 저는 2급 장애인 입니다. 다름이아니라 괜찮은 장애인협회...,"지식iN의 0.1% ,파워지식iN입니다~ kod****님의 입장에서, 과장되지...",2012.09.07
...,...,...,...
9378,\n\t\t\t\t\t\t 장애인 대출 가능 한지 묻고 싶습니다 .... 남자구요 ...,농x중앙회 소속 캐피탈 대출상담사 입니다.장애인여부는 아무 문제없으시구요일단 자세한...,2012.05.07
9379,"\n\t\t\t\t\t\t이우학교에 대해서 알아보려고 하는데요,이우학교는 장애인학교...",장애인학교 아닙니다. 대안학교 입니다신애라씨 아들이 다니고 있는 대안학교 이죠 등록...,2012.08.17
9380,\n\t\t\t\t\t\t안녕하세요 많은조언 해결방안좀 부탁 드립니다 우선 가족 은...,영세민이신경우 각 동사무소에 요청을하시면 국민보금자리로 옮기실수있으십니다 그집주인....,2012.09.26
9381,\n\t\t\t\t\t\t 어머니께서 2009년 1월 갑상선암 전절제 수술을 받으셨...,공제 대상이 되신다면 소급적용 받을 수 있습니다.국세기본법제45조의2(경정 등의 청...,2012.12.18


In [68]:
naver_2012.to_csv('2012장애인.csv')

In [63]:
naver_2013 = naver_crawler('장애인', 2013)
naver_2013

,Question,Answer,Date
0,\n\t\t\t\t\t\t 이번에 LPG차량을 구입했습니다. 장애인 명의는 아니구요...,1. 지금 장애인복지카드란게 발급 가능한가요???? 해당된다면 보호자 명의로 가...,2013.12.07
1,"\n\t\t\t\t\t\t 오는 10월부터 장애인등급심사를 완화한다 하였는데, 여기...",질문하신 내용은 국가장애 (복지카드) 심사를 완화한 것 입니다. 장애등급심사 과...,2013.09.29
2,\n\t\t\t\t\t\t 1)휠체어를 탄 제2의아인슈타인은?2)말은 잘 못하지만 ...,1)휠체어를 탄 제2의 아인슈타인은?=스티븐 호킹 박사 2)말은 잘 못하지만 거짓...,2013.04.16
3,\n\t\t\t\t\t\t장애인 행정도우미가 하는 일이 뭔가요?월급이랑 하는 일을 ...,안녕하세요. 힘이 되는 평생친구 보건복지부입니다.귀하의 건강과 행복을 기원합니다. ...,2013.06.18
4,"\n\t\t\t\t\t\t 장애인 증명서를 발급해야 하는데요, 대상자가 제가 아니고...",안녕하세요~ 국민과 함께하는 정부대표전화 110 지식파트너입니다. 질문하신 장애인증...,2013.08.29
...,...,...,...
70,\n\t\t\t\t\t\t안녕하세요 짧게 말하겠습니다 장애인분이 동내에 한분 계신데...,장애인 이라고 차별을 하셨군요?배려는 좋으나 신고 하지 않은 것은 인권 차별이죠....,2013.04.15
71,\n\t\t\t\t\t\t 안녕하세요. 현재 저는 만25세의 여성으로 5년 전...,1) 저는 현재 지방에 거주하고 있고 정기검진은 서울의 2차 안과병원에서 받고 있습...,2013.06.11
72,\n\t\t\t\t\t\t 안녕하세요. 현재 저는 만25세의 여성으로 5년 전...,1) 저는 현재 지방에 거주하고 있고 정기검진은 서울의 2차 안과병원에서 받고 있습...,2013.06.11
73,\n\t\t\t\t\t\t 장애인이 법률을 위반하였을때 감형(벌받을수)될수있나요??...,"장애인, 비장애인 모두 차별없이 동등한 대우를 받으며 따라서 법정 처벌또한 동등합니...",2013.07.30


In [64]:
naver_2013.to_csv('2013장애인.csv')

In [69]:
naver_2014 = naver_crawler('장애인', 2014)
naver_2014

,Question,Answer,Date
0,\n\t\t\t\t\t\t 안녕하세요.다름이 아니라 신차구매시에 장애인 혜택중 하나...,현재 기아자동차에서 판매하고 있는 장애인용 차량은 아래와 같습니다. 2014년형 ...,2014.05.01
1,\n\t\t\t\t\t\t 안녕하세요.다름이 아니라 신차구매시에 장애인 혜택중 하나...,현재 기아자동차에서 판매하고 있는 장애인용 차량은 아래와 같습니다. 2014년형 ...,2014.05.01
2,\n\t\t\t\t\t\t장애인연금 때문에 질문 올립니다.7월부터 장애인연금이 변동...,안녕하세요. 힘이 되는 평생친구 보건복지부입니다.귀하의 건강과 행복을 기원합니다. ...,2014.06.27
3,\n\t\t\t\t\t\t어머니는 장애인 3급 판정을 받으셨어요... 1. 승용차...,"기아자동차 경차 및 장애인용 차량 구매 시, 제공받을 수 있는 혜택 및 절차는 아...",2014.06.08
4,\n\t\t\t\t\t\t어머니는 장애인 3급 판정을 받으셨어요... 1. 승용차...,"기아자동차 경차 및 장애인용 차량 구매 시, 제공받을 수 있는 혜택 및 절차는 아...",2014.06.08
...,...,...,...
2855,내용없음,NaN,NaN
2856,내용없음,NaN,NaN
2857,내용없음,NaN,NaN
2858,내용없음,NaN,NaN


In [70]:
naver_2014.to_csv('2014장애인.csv')

In [71]:
naver_2015 = naver_crawler('장애인', 2015)
naver_2015

,Question,Answer,Date
0,\n\t\t\t\t\t\t장애인화장실 공사를 하려는데장애인화장실규격이 어떻게 되는지...,장애인 화장실 세부기준관련법령 : 장애인 노인 임산부등의편의증진보장에관한법률장애인등...,2015.11.11
1,\n\t\t\t\t\t\t장애인화장실 공사를 하려는데장애인화장실규격이 어떻게 되는지...,장애인 화장실 세부기준관련법령 : 장애인 노인 임산부등의편의증진보장에관한법률장애인등...,2015.11.11
2,\n\t\t\t\t\t\t안녕하세요?만약에 장애인 주차구역에 차량 번호가 다른 장애...,만약에 장애인 주차구역에 차량 번호가 다른 장애인 주차증을 가지고 주차했다면 어쩧게...,2015.11.04
3,\n\t\t\t\t\t\t음. . . . .학교 숙제라서요. . . 빠르게 알려주시...,안녕하세요.지식인 neverdiearea 입니다.1. 시각장애인은 운전면허를 딸수있...,2015.11.03
4,\n\t\t\t\t\t\t1.전국 장애인의수는 몇명일까요.?2. 선천적 장애인의수는...,"1. 전국 장애인의수는 몇 명일까요.?= 2013년 말 기준으로 2,501,112명...",2015.04.19
...,...,...,...
2897,\n\t\t\t\t\t\t제가 학교 숙제로 시각장애인 편으시설 중에서 횡단보도 에서...,먼저 곡선인 부분보다 고쳐야겠죠!!!!!!!! 그 부분이 방향이 이상해서요... 시...,2015.03.21
2898,내용없음,장애인되요,2015.08.10
2899,\n\t\t\t\t\t\t일단 처음에는 파워레인저같은거나오고 남자주인공은 잘생기고 ...,윈드미라클의 바람동화드라마|총 1부작|전체 관람가|2015.04.20.~2015.0...,2015.12.15
2900,\n\t\t\t\t\t\t저희가 다음주 화요일에 장애인을 위한 과학의 대해 토론 대...,먼저 장애인을 위한 발명품을 구상하는 것이 좋습니다.그 후 겉모습을 소개하고(포토샵...,2015.04.04


In [72]:
naver_2015.to_csv('2015장애인.csv')

In [73]:
naver_2016 = naver_crawler('장애인', 2016)
naver_2016

,Question,Answer,Date
0,\n\t\t\t\t\t\t안녕하세요. 아무리 검색해도 확실한 답을 못찾겠어서 질문드...,"장애인특별공급에 대해서는 한국토지주택공사 공급분이든, 지방자치단체나 민간업체 공급분...",2016.10.31
1,\n\t\t\t\t\t\t1심에서 장애인위계등 간음으로 징역5년 받고 구속되어현재 ...,감사합니다. 로시컴과 함께 지식iN 법률상담을 진행하고 있는 송명호 변호사입니다.우...,2016.11.21
2,\n\t\t\t\t\t\t1심에서 장애인위계등 간음으로 징역5년 받고 구속되어현재 ...,감사합니다. 로시컴과 함께 지식iN 법률상담을 진행하고 있는 송명호 변호사입니다.우...,2016.11.21
3,\n\t\t\t\t\t\t장애인용 화장실 그림과 글로 설명좀해주새요.두산백과 처럼요...,장애인등의 이용이 가능한 화장실 가. 일반사항 (1) 설치장소 (가...,2016.10.13
4,\n\t\t\t\t\t\t장애인용 화장실 그림과 글로 설명좀해주새요.두산백과 처럼요...,장애인등의 이용이 가능한 화장실 가. 일반사항 (1) 설치장소 (가...,2016.10.13
...,...,...,...
1604,내용없음,NaN,NaN
1605,내용없음,NaN,NaN
1606,내용없음,NaN,NaN
1607,내용없음,NaN,NaN


In [74]:
naver_2016.to_csv('2016장애인.csv')

In [75]:
naver_2017 = naver_crawler('장애인', 2017)
naver_2017

,Question,Answer,Date
0,내용없음,■ 장애등급 1급~6급은 고속도로 통행료 50% 할인이 가능하며 이용방법은 다음과...,2017.11.25
1,\n\t\t\t\t\t\t장애인 등록증 분실해서 재발급을받으려고 합니다혹시 비용이 ...,무료입니다.다만 등기로 받으실 경우 요금이 발생할 수 있습니다.,2017.12.07
2,\n\t\t\t\t\t\t지역사회재활사업이라는게 정확히 무엇인가요?스스로 살아갈수 ...,장애인복지실천에서 수행되는 재활을 서비스전달방식의 측면에서 살펴보면 2가지로 구분하...,2017.12.06
3,내용없음,안녕하세요. 광주전남중소벤처기업청 비즈니스지원단 상담위원입니다.장애인기업이란 장애인...,2017.11.10
4,\n\t\t\t\t\t\t한달전 세이클럽에서 채팅으로 알게됬고 자긴 모텔에 있으니 ...,안녕하세요. 서울지방변호사회-네이버 지식iN 상담변호사 이재호 입니다. 성폭력 범죄...,2017.11.03
...,...,...,...
7154,"\n\t\t\t\t\t\t저는 5년쯤전에 장애등급을 받고 인생 포기하고 살다가,작년...",안녕하세요 학습플래너 쑤욱쌤입니다~^^♥제 생각에는 장애인과 어울리는 직업이라고 하...,2017.08.26
7155,\n\t\t\t\t\t\t장애인주차장에 주차를 하는 사진을 누가 1gb급 첨부파일로...,장애인 주차구역에 주차는 불법으로 선처는 없습니다.부과받으신 과태료 납부하셔야 합니다.,2017.10.20
7156,\n\t\t\t\t\t\t20남 입니당장애우나 장애인 비장애인이랑 다른은 아니고 음...,"'장애우'는 친구라는 의미를 가진 한자인 벗 우(友)자를 써서 만든 말입니다.헌데,...",2017.05.08
7157,\n\t\t\t\t\t\t장애 5등급입니다.현재 경유 2000cc 미만 차량 1대로...,안됩니다.lpg 차량을 장애인 차량으로 등록해야 하기때문에 경유 차량은 혜택 받을 ...,2017.05.14


In [76]:
naver_2017.to_csv('2017장애인.csv')

In [77]:
naver_2018 = naver_crawler('장애인', 2018)
naver_2018

,Question,Answer,Date
0,\n\t\t\t\t\t\t안녕하세요 현재 사회복무요원을 진행중인 20대입니다. 저는...,# 로톡 | 박중광 변호사 [변호사 프로필보기]# 사무실 문의 전화: 050-772...,2018.11.26
1,\n\t\t\t\t\t\t 1.연말정산때 장애인증명서를 함께 제출하면 환급되는 정확...,장애가 있는 경우 소득세액을 환급하는 금액이 아니라 소득공제되는 금액이 기본공제 1...,2018.11.06
2,\n\t\t\t\t\t\t국가장학금 선정기준이 소득분위에 따른 분류뿐인가요?제가 장...,안녕하세요^^한국장학재단입니다. 문의하신 국가장학금 에 대한 답변입니다.국가장학금...,2018.11.22
3,\n\t\t\t\t\t\t장애인 이면서 수급자 수도요금 할인 얼마나????3식구 서...,안녕하세요. 서울생활 행복도우미 120다산콜센터 지식파트너입니다.◈ 서울시 국민기초...,2018.11.07
4,\n\t\t\t\t\t\t공무원 중증장애인 중복장애인도 응시가능한지요? 장애인 구분...,안녕하세요. 답변 드리겠습니다. 중증장애인도 해당이 되지만 보건복지부 및 국가보훈처...,2018.12.08
...,...,...,...
3805,내용없음,"-.장애1-3급은 개별소비세(6.5%),취등록세(7%),공채(4-20%),매년 내년...",2018.11.22
3806,내용없음,NaN,NaN
3807,내용없음,NaN,NaN
3808,내용없음,NaN,NaN


In [78]:
naver_2018.to_csv('2018장애인.csv')

In [79]:
naver_2019 = naver_crawler('장애인', 2019)
naver_2019

,Question,Answer,Date
0,\n\t\t\t\t\t\t안녕하세요한국장애인고용공단에 취업을 목표로 하는 취준생입니...,...,2018.07.26
1,\n\t\t\t\t\t\t안녕하세요한국장애인고용공단에 취업을 목표로 하는 취준생입니...,...,2018.07.26
2,\n\t\t\t\t\t\t시청각 장애인을 위한 시청각 장애인용 TV를 본 적이 있습...,...,2019.09.26
3,\n\t\t\t\t\t\t사회복지학과 학생입니다. 과제 중 장애인기업을 지원할 수 ...,...,2019.09.27
4,\n\t\t\t\t\t\t윌슨병을 가지고 있어 20년 이상 알타민등을 복용중입니다....,...,2019.10.30
...,...,...,...
6610,\n\t\t\t\t\t\t뇌병변 1급 장애인이 장애인 인식개선 강사 자격증을 따고자...,...,2019.07.16
6611,내용없음,NaN,NaN
6612,내용없음,NaN,NaN
6613,내용없음,NaN,NaN


In [80]:
naver_2019.to_csv('2019장애인.csv')

In [81]:
naver_df = pd.concat([naver_2002, naver_2003], axis=0)
naver_df = pd.concat([naver_df, naver_2004], axis=0)
naver_df = pd.concat([naver_df, naver_2005], axis=0)
naver_df = pd.concat([naver_df, naver_2006], axis=0)
naver_df = pd.concat([naver_df, naver_2007], axis=0)
naver_df = pd.concat([naver_df, naver_2008], axis=0)
naver_df = pd.concat([naver_df, naver_2009], axis=0)
naver_df = pd.concat([naver_df, naver_2010], axis=0)
naver_df = pd.concat([naver_df, naver_2011], axis=0)
naver_df = pd.concat([naver_df, naver_2012], axis=0)
naver_df = pd.concat([naver_df, naver_2013], axis=0)
naver_df = pd.concat([naver_df, naver_2014], axis=0)
naver_df = pd.concat([naver_df, naver_2015], axis=0)
naver_df = pd.concat([naver_df, naver_2016], axis=0)
naver_df = pd.concat([naver_df, naver_2017], axis=0)
naver_df = pd.concat([naver_df, naver_2018], axis=0)
naver_df = pd.concat([naver_df, naver_2019], axis=0)
naver_df

,Question,Answer,Date
0,"\n\t\t\t\t\t\t아주 어렸을 때부터 정말 궁금했습니다..시각장애인들, 즉 ...","'꿈'이라는건 정확히 알려진 바가 없으나""잠재의식(공포,과거의 경험,소망),이 깊지...",2002.10.29
1,\n\t\t\t\t\t\t장애인들이 개선 되기를 원하는 점은 무엇일까요..?여러가지...,안녕하세요 균이 입니다 *^^*장애인들이 가장 개선 되기 원하는 것은 장애인에 대...,2002.11.13
2,\n\t\t\t\t\t\t제가 아침에 환청을 듣고 잠에서 깼거든요;너무 무서워서 생...,먼저 환청이라는 것은 주위에 사람이나 소리나는 사물이 아무것도 없는데도 어떤 소리나...,2002.12.31
3,\n\t\t\t\t\t\tTV 연속극에서 시각장애인이 눈물을 흘리는 장면이 나와 이...,눈물은 기본적인 눈물과 반사적인 눈물로 나누어 볼 수 있는데 우리가 일반적으로 아는...,2002.12.25
4,\n\t\t\t\t\t\t그냥 비장애인들도 하는 경기 말구요...장애인들만 하는 경...,1996년 애틀란타 장애인올림픽에서 시범종목이었던 휠체어 럭비와 요트 경기가 정식 ...,2002.10.30
...,...,...,...
6610,\n\t\t\t\t\t\t뇌병변 1급 장애인이 장애인 인식개선 강사 자격증을 따고자...,...,2019.07.16
6611,내용없음,NaN,NaN
6612,내용없음,NaN,NaN
6613,내용없음,NaN,NaN


In [82]:
print(len(naver_df))

75254


In [83]:
import pandas
naver_df.to_csv('naver지식인_장애인2002-2019_new.csv', encoding = 'utf-8-sig')

# 간단 명료하게 해보자.

In [1]:
import time
import urllib.request, urllib.parse
from bs4 import BeautifulSoup
import csv
import re
import pandas as pd
import time
import numpy as np
import requests
import lxml.html
import random

In [8]:
def naver_crawler(quest, year):
    # 검색키워드, 검색 날짜 범위 지정 
    query = quest
    period_from =  str(year) + '.01.01.'
    period_to =  str(year) + '.12.31.'
    
    # 주소 생성 
    naver_front = 'https://kin.naver.com/search/list.nhn?query='
    period_from = '&period=' + str(period_from)
    period_to = '%7c' + str(period_to)
    kin_url = naver_front + query + period_from + period_to + '&page={}'
    
    # 크롤링 내역 담을 리스트 생성 
    ques = []
    answer = []
    date = []
    
    # 각 범위주소마다 페이지 범위 넘겨가며 크롤링 진행 
    page = 0
    while True:
        page += 1 
        # 점속 컨택 
        # time.sleep(0.1)
        res = requests.get(kin_url.format(page)) 
        root = lxml.html.fromstring(res.text)
        css = root.cssselect('.basic1 a') # 링크가 있는 루트 
     
        # 각 질문마다의 링크 리스트에 담음. 
        links = []
        # 링크가 이상한 경우가 있어 https 주소로 되어있는 경우만 가져옴.
        for i in css:
            if 'https:' in i.attrib['href']:
                links.append(i.attrib['href'])
            else:
                next
                
        # 각 링크마다의 제목, 답변, 게시일 담음.
        for link in links:
            #time.sleep(0.1)
            res_links = requests.get(link)
            root_links = lxml.html.fromstring(res_links.text) 
            try:
                ques.append(root_links.cssselect('.c-heading__content')[0].text_content())
            except IndexError:
                ques.append('내용없음')
            # time.sleep(0.1)
            try:
                answer.append(root_links.cssselect('._endContentsText')[0].text_content())
            except IndexError:
                answer.append(np.NaN)
            # time.sleep(0.1)
            try:
                date.append(root_links.cssselect('.c-userinfo__info')[0].text_content()[3:])
            except IndexError:
                date.append(np.NaN)  # 게시글이 없는 경우가 있음 결측값 처리 
        if len(links) == 0: 
            break # links가 없다. 즉 페이지에 질문이 없는 경우는 페이지가 끝난 것이므로 멈춤
        
    DF = pd.DataFrame({'Question':ques,'Answer':answer,'Date':date}) 
    return DF 

In [3]:
df = pd.DataFrame({'Question':[0],'Answer':[0],'Date':[0]})

def concat_year(x):
    df2 = pd.concat([df, naver_crawling('장애인',x)], axis=0)
    return df2 

In [5]:
for i in range(2002,2020): 
    naver_final = concat_year(i) 
    time.sleep(1)
    print(i, len(naver_final))
    
naver_final.drop([0], inplace=True) 
print(naver_final) 

2002 349
2003 5641
2004 1
2005 1
2006 1
2007 1
2008 1
2009 1
2010 1
2011 1
2012 1
2013 1
2014 1
2015 1
2016 1
2017 1
2018 1
2019 1
Empty DataFrame
Columns: [Question, Answer, Date]
Index: []


In [ ]:
naver_final.shape

In [15]:
import pandas
naver_final.to_csv('naver_final_장애인2002-2019.csv', encoding = 'utf-8-sig')

NameError: name 'naver_final' is not defined